# CS771A Assignment 1
\- Yash Gupta (190997)

## Q1. Gradient Descent

In [36]:
# importing libraries
import numpy as np
import time as t

In [16]:
# defining the gradient descent function
def gradient_descent(gradient, init_, learn_rate, n_iter=50, tol=1e-06):
    x = init_
    for _ in range(n_iter):
        delta = -learn_rate * gradient(x)
        if np.all(np.abs(delta) <= tol):
            break
        x += delta
    return round(x * 1000) / 1000

## Q1. (a)
Use this function to find minima for (i) $x^2 + 3x + 4$ and (ii) $x^4 – 3x^2 + 2x$.

First, we'll have to find the gradient of the expressions in (i) and (ii). As these are expressions in one variable, their gradients are simply there derivatives wrt. x. They can be given as follows:  
(i) $2x + 3$  
(ii) $4x^3 - 6x + 2$

Now that we have the gradients, we can find the minima as follows:

### (i)

In [141]:
min_x1 = gradient_descent(gradient=lambda x: 2 * x + 3, init_=0.0, learn_rate=0.1)
print('Point of minima:', min_x1)
min_y1 = min_x1 ** 2 + 3 * min_x1 + 4
print('Minima:', min_y1)

Point of minima: -1.5
Minima: 1.75


### (ii)

In [140]:
min_x2 = gradient_descent(gradient=lambda x: 4 * (x ** 3) - 6 * x + 2, init_=0.0, learn_rate=0.1)
print('Point of minima:', min_x2)
min_y2 = min_x2 ** 4 - 3 * (min_x2 ** 2) + 2 * min_x2
print('Minima:', min_y2)

Point of minima: -1.366
Minima: -4.848076206064


Hence, 

## Q1. (b)
Write a gradient function to calculate gradients for a linear regression $y = ax + b$

The loss function ($L_2$ loss) for a linear regression $y = ax + b$, say $L(a, b)$, will be: 
$$ L(a, b) = \sum_{n = 1}^N (y_n - a x_n - b)^2 $$
where N is the number of data points.

The gradient for this loss function will be:
$$ \left[ \frac{\partial L}{\partial a}, \frac{\partial L}{\partial b} \right] = \left[ -2 \sum_{n = 1}^N x_n (y_n - a x_n - b), -2 \sum_{n = 1}^N (y_n - a x_n - b) \right] $$

Hence, the gradient function can be written as follows:

In [20]:
def gradient_lr(params):
    grad = np.array([0.0, 0.0])
    for Xn, yn in zip(X, y): # X is the features and y is the labels both of which will be global variables
        grad[0] += -2 * Xn * (yn - params[0] * Xn - params[1])
        grad[1] += -2 * (yn - params[0] * Xn - params[1])
    return grad

## Q1. (c)
Generate artificial data for this regression according to the following protocol and use gradient descent to find the optimal parameters relating X with y.

In [82]:
np.random.seed(0)
X = 2.5 * np.random.randn(10000) + 1.5 # array of 10000 values with mean = 1.5, stddev = 2.5
res = 1.5 * np.random.randn(10000) # generate 10000 residual terms
y = 2 + 0.3 * X + res # actual values of y

Now, we need to make a small modification in the gradient_descent() function defined earlier to support multivariate gradient descent. 

In [106]:
# defining the gradient descent function
def gradient_descent_multivariate(gradient, init_, learn_rate, n_iter=50, tol=1e-06):
    x = init_
    for _ in range(n_iter):
        delta = -learn_rate * gradient(x)
        if np.all(np.abs(delta) <= tol):
            break
        x += delta
    return np.round(x, 3) # Use np.round() instead of round() to round a numpy array

Running gradient descent on this data:

In [117]:
params_init = np.array([0.0, 0.0])
tic = t.process_time()
params = gradient_descent_multivariate(gradient_lr, init_=params_init, learn_rate=0.00001)
toc = t.process_time()
time_vanilla_gd = toc - tic
print('Time taken:', time_vanilla_gd, 'seconds')
params

Time taken: 1.044782304999984 seconds


array([0.295, 2.022])

## Q1. (d)
Implement minibatch stochastic gradient descent using the code base you have developed so far.

We can define the minibatch stochastic gradient descent function by creating a minibatch in every iteration of the gradient descent and passing the minibatch to the gradient function as follows:

In [33]:
# defining the minibatch stochastic gradient descent function
def minibatch_sgd(gradient, init_, learn_rate, n_iter=50, tol=1e-06, batch_size=125):
    x = init_
    for _ in range(n_iter):
        idx = np.random.choice(np.arange(len(X)), batch_size, replace=False) # indexes of the minibatch
        X_batch = X[idx]
        y_batch = y[idx]
        delta = -learn_rate * gradient(x, X_batch, y_batch)
        if np.all(np.abs(delta) <= tol):
            break
        x += delta
    return np.round(x, 3)

We will also need to define a gradient function which takes the minibatches as inputs as follows:

In [34]:
def gradient_lr_sgd(params, X_batch, y_batch):
    grad = np.array([0.0, 0.0])
    for Xn, yn in zip(X_batch, y_batch):
        grad[0] += -2 * Xn * (yn - params[0] * Xn - params[1])
        grad[1] += -2 * (yn - params[0] * Xn - params[1])
    return grad

Running minibatch stochastic gradient descent on the data:

In [118]:
params_sgd_init = np.array([0.0, 0.0])
tic = t.process_time()
params_sgd = minibatch_sgd(gradient_lr_sgd, init_=params_sgd_init, learn_rate=0.00001)
toc = t.process_time()
time_sgd = toc - tic
print('Time taken:', time_sgd, 'seconds')
params

Time taken: 0.032973420000018905 seconds


array([0.295, 2.022])

## Q1. (e)
Does SGD do better or worse in terms of time performance on our data? Is there an optimal minibatch size that works best? Quantify and interpret your findings.

Let's look at the times taken by vanilla gradient descent and minibatch SGD.

In [119]:
print('Time taken by vanilla gradient descent:', time_vanilla_gd)
print('Time taken by minibatch SGD:', time_sgd)

Time taken by vanilla gradient descent: 1.044782304999984
Time taken by minibatch SGD: 0.032973420000018905


As we can see, minibatch SGD does much better in terms of time performance than vanilla gradient descent. 

Now, let's find the optimal minibatch size that works best.

In [139]:
params_sgd_init = np.array([0.0, 0.0])
min_time = float('inf')
opt_batch_size = 0
for batch_size in range(1, 1001, 125):
    # print('Current batch size:', batch_size)
    tic = t.process_time()
    params_sgd = minibatch_sgd(gradient_lr_sgd, init_=params_sgd_init, learn_rate=0.00001)
    toc = t.process_time()
    time_sgd = toc - tic
    if time_sgd < min_time:
        min_time = time_sgd
        opt_batch_size = batch_size
print('The optimal minibatch size is', opt_batch_size, 'and it takes', min_time, 'seconds')

The optimal minibatch size is 751 and it takes 0.019864095000002635 seconds


In [74]:
# Interpret your findings

## Q2. Bayesian network

![Bayesian Network](bayesian_network.png "Bayesian Network")

## Q2. (i)
Calculate the probability that someone has both cold and a fever

Let $C$ represent cold and $F$ represent fever. So, we need to find $P(C \cap F)$. 
$$ P(C \cap F) = P(F | C) P(C) = 0.307 \times 0.02 = 0.00614 $$
Hence, the probability that someone has both cold and a fever is 0.00614

## Q2. (ii)
Calculate the probability that someone who has a cough has a cold

Let $X$ represent cough, $C$ represent cold, $L$ represent lung disease and $S$ represent smokes. So, we need to find $P(C | X)$. 
$$ P(C | X) = \frac{P(X | C) P(C)}{P(X)} $$
Now, $$ P(X | C) = P \bigl( X | (C \cap L) \bigr) P(L | C) + P \bigl( X | (C \cap \overline{L}) \bigr) P(\overline{L} | C) $$
Since C and L are independent, $P(L | C) = P(L)$ and $P(\overline{L} | C) = P(\overline{L})$.
So, $$ P(X | C) = P \bigl( X | (C \cap L) \bigr) P(L) + P \bigl( X | (C \cap \overline{L}) \bigr) P(\overline{L}) $$
Now, $$ P(L) = P(L | S) P(S) + P(L | \overline{S}) P(\overline{S}) = 0.1009 \times 0.2 + 0.001 \times 0.8 = 0.02098 $$
and $$ P(\overline{L}) = 1 - P(L) = 1 - 0.02098 = 0.97902 $$
Hence, $$ P(X | C) = 0.7525 \times 0.02098 + 0.505 \times 0.97902 = 0.51019255 $$
Now, $$ P(X) = P \bigl(X | (L \cap C) \bigr) P(L \cap C) + P \bigl(X | (L \cap \overline{C}) \bigr) P(L \cap \overline{C}) + P \bigl(X | (\overline{L} \cap C) \bigr) P(\overline{L} \cap C) + P \bigl(X | (\overline{L} \cap \overline{C}) \bigr) P(\overline{L} \cap \overline{C}) $$
Again, since L and C are independent, $P(L \cap C) = P(L) P(C)$,  $P(L \cap \overline{C}) = P(L) P(\overline{C})$, $P(\overline{L} \cap C) = P(\overline{L}) P(C)$ and $P(\overline{L} \cap \overline{C}) = P(\overline{L}) P(\overline{C})$  
Hence, $$ P(X) = P \bigl(X | (L \cap C) \bigr) P(L) P(C) + P \bigl(X | (L \cap \overline{C}) \bigr) P(L) P(\overline{C}) + P \bigl(X | (\overline{L} \cap C) \bigr) P(\overline{L}) P(C) + P \bigl(X | (\overline{L} \cap \overline{C}) \bigr) P(\overline{L}) P(\overline{C}) $$
$$ = 0.7525 \times 0.02098 \times 0.02 + 0.505 \times 0.02098 \times 0.98 + 0.505 \times 0.97902 \times 0.02 + 0.01 \times 0.97902 \times 0.98 = 0.030181249 $$
Hence, $$ P(C | X) = \frac{0.51019255 \times 0.02}{0.030181249} = 0.33808577637 $$

Hence, the probability that someone who has a cough has a cold is 0.33808577637

## Q3. Derive the MLE for the parameters of a k-sided multinomial distribution.

Suppose there are $k$ categories. Let us do $n$ independent trials, such that each trial leads to the success of exactly one of the $k$ categories. Let $p_i$ be the probability of success and $X_i$ be the number of successes of category $i$ for $i = 1, \ldots, k$.  
Then, the likelihood of the k-sided multinomial distribution with parameters n and $p_i$ for $i = 1, \ldots, k$ is:
$$ f(\textbf{x}; n, \textbf{p}) = f(x_1, \ldots, x_k; n, p_1, \ldots, p_k) = n! \prod_{i = 1}^k \frac{p_i^{x_i}}{x_i!} $$
where $\sum_{i = 1}^{k} x_i = n$ and $\sum_{i = 1}^{k} p_i = 1$. 

We will write $f(\textbf{x}; n, \textbf{p})$ as $f(\textbf{p})$ for brevity. 
 
Now, the log-likelihood will be:
$$ LL(\textbf{p}) = \log f(\textbf{p}) = \log \left( n! \prod_{i = 1}^k \frac{p_i^{x_i}}{x_i!} \right) $$
$$ = \log n! + \log \prod_{i = 1}^k \frac{p_i^{x_i}}{x_i!} $$
$$ = \log n! + \sum_{i = 1}^k \log \frac{p_i^{x_i}}{x_i!} $$
$$ = \log n! + \sum_{i = 1}^k x_i \log p_i - \sum_{i = 1}^k \log x_i! $$

We need to maximize $LL(\textbf{p})$ with the constraint $\sum_{i = 1}^{k} p_i = 1$. 

Hence, we can define the lagrangian as:
$$ l(\textbf{p}, \lambda) = LL(\textbf{p}) + \lambda \left( 1 - \sum_{i = 1}^k p_i \right) $$
where $\lambda$ is the lagrange multiplier. 

Now, to find $\argmax_{\textbf{p}} l(\textbf{p}, \lambda)$, 
$$ \frac{\partial}{\partial p_i} l(\textbf{p}, \lambda) = 0 $$
for $i = 1, \ldots, k$

$$ \frac{\partial}{\partial p_i} l(\textbf{p}, \lambda) = \frac{\partial}{\partial p_i} LL(\textbf{p}) + \frac{\partial}{\partial p_i} \lambda \left( 1 - \sum_{i = 1}^k p_i \right) $$
$$ = \frac{\partial}{\partial p_i} \sum_{i = 1}^k x_i \log p_i - \lambda \frac{\partial}{\partial p_i} \sum_{i = 1}^k p_i $$
$$ = \frac{x_i}{p_i} - \lambda $$

Since, $\frac{\partial}{\partial p_i} l(\textbf{p}, \lambda) = 0$, 
$$ \frac{x_i}{p_i} - \lambda = 0 $$
$$ \Rightarrow p_i = \frac{x_i}{\lambda} $$

Now, since $\sum_{i = 1}^{k} p_i = 1$, 
$$ \sum_{i = 1}^{k} \frac{x_i}{\lambda} = 1 $$
$$ \Rightarrow \lambda = \sum_{i = 1}^{k} x_i = n $$

Hence, 
$$ p_i = \frac{x_i}{n} $$

In other words, 
$$ \textbf{p} = \left( \frac{x_1}{n}, \ldots, \frac{x_k}{n} \right) $$
is the MLE for the parameters of a k-sided multinomial distribution. 